In [1]:
cd E:/GitHub/computervision

E:\GitHub\computervision


In [2]:
import torch
import os
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

from torch.nn import CrossEntropyLoss
from datetime import datetime
from torch.optim import lr_scheduler, SGD, Adamax
from modules.training_util import PytorchDataset, PytorchTraining, EarlyStopping
from modules.models_util import DenseNet

In [3]:
# Variables for continuing previous training
CONTINUE_TRAINING_DIR = None
# CONTINUE_TRAINING_DIR = 'notebooks/vit/models/'

MODEL_WEIGHTS_PATH = None

if CONTINUE_TRAINING_DIR:
    MODEL_WEIGHTS_PATH = os.path.join(CONTINUE_TRAINING_DIR,'epoch_15.pth')

# CONFIG VARIABLES
FULLY_TRAIN = False
base_path = 'E:/Datasets/cor-splits'
DATA_DIR = [
    # os.path.join(base_path,'sgkf-8-1-1-size-540'),
    # os.path.join(base_path,'underperforming_cameras_96'),
    # os.path.join(base_path,'underperforming_cameras_15'),
    # os.path.join(base_path,'underperforming_cameras_30'),
    os.path.join(base_path,'underperforming_cameras_60_v2'),
    # os.path.join(base_path,'underperforming_cameras_61_15img'),
    # os.path.join(base_path,'underperforming_cameras_59_45imgs'),
    # os.path.join(base_path,'random_cameras_61_30imgs'),
    # os.path.join(base_path,'random_cameras_59_45imgs'),
    # os.path.join(base_path,'random_cameras_71_15img'),
    # os.path.join(base_path,'underperforming_cameras_60_v2_euro'),
    # os.path.join(base_path,'underperforming_cameras_60_v2_itsc'),
    # os.path.join(base_path,'sgkf-8-2-size-1010'),
    # os.path.join(base_path,'natural'),
    # os.path.join(base_path,'balanced_positive'),
    # os.path.join(base_path,'natural_positive'),
    # os.path.join(base_path,'balanced'),
    # 'E:/Datasets/itsc_flood_dataset'
    ]
START_EPOCH = 1 # Change it from 1 if you're continuing an existing run
N_EPOCHS = 50
BATCH_SIZE = 16
SAVE_INTERVAL = 10 # How many epochs between each backup
## Learning Rate Scheduler
LR = [1e-03,1e-04,1e-05]  # When continuing a existing run, update it to the last epoch LR value
LR_EPOCH_SCHEDULE = 5  # Number of epochs before altering LR
LR_GAMMA = 0.1
## Early Stopping
PATIENCE = 10
MIN_DELTA = 0
MIN_EPOCH = 10 # Number of epochs before starting patience counter
##
BP = [0.5]

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
archs = ['169']

In [5]:
for arch in archs:
    for dir in DATA_DIR:
        for lr in LR:
            for brightness in BP:
                model_arch = DenseNet(architecture=arch,bp=brightness)
                
                model = model_arch.load(MODEL_WEIGHTS_PATH, FULLY_TRAIN)
                model = model.to(device)
                
                criterion = CrossEntropyLoss()

                if FULLY_TRAIN:
                    params = model.parameters()
                else:
                    params = model.classifier.parameters()
                
                optimizer = SGD(params, lr=lr)
                # optimizer = Adamax(model.parameters(), lr=LR)

                step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=LR_EPOCH_SCHEDULE, gamma=LR_GAMMA)
                
                es = EarlyStopping(patience=PATIENCE, 
                                delta=MIN_DELTA, 
                                min_epoch=MIN_EPOCH, 
                                starting_patience_counter=0) # change it from zero only if continuing an existing run with patience counter != 0
                
                dataset = PytorchDataset(dir, model_arch.data_transforms, BATCH_SIZE)

                working_folder = 'notebooks/densenet'
                models_folder = 'models'

                curr_time = datetime.now()
                curr_time = curr_time.strftime("%Y-%m-%d %H-%M-%S")

                output_folder = os.path.join(working_folder, models_folder)
                output_folder = os.path.join(output_folder, curr_time)
            
                if CONTINUE_TRAINING_DIR:
                    output_folder = CONTINUE_TRAINING_DIR
                    
                if not os.path.exists(output_folder):
                    os.makedirs(output_folder)

                trainer = PytorchTraining(device, dataset, output_folder)
                model = trainer.train_pytorch_model(model, criterion, optimizer, step_lr_scheduler, es, START_EPOCH, N_EPOCHS, SAVE_INTERVAL, arch)

c:\Users\Adm\anaconda3\envs\computervision\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet169_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet169_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Only fully connected layer requires grad.
Saving model to folder notebooks/densenet\models\2024-11-18 15-10-49

Model DenseNet 169
Fully Trained = False
Dataset E:/Datasets/cor-splits\underperforming_cameras_60_v2
Learning Rate Epoch Schedule = 5
Learning Rate Gamma = 0.1
Optimizer = SGD
Batch Size = 16


Epoch 1/50
----------

train Loss: 0.6852 Acc: 0.5467
val Loss: 0.5967 Acc: 0.7137
Epoch duration: 0 m 44s
Learning Rate = 0.001



Epoch 2/50
----------

train Loss: 0.6310 Acc: 0.6526
val Loss: 0.5343 Acc: 0.7618
Epoch duration: 0 m 43s
Learning Rate = 0.001



Epoch 3/50
----------

train Loss: 0.5973 Acc: 0.6887
val Loss: 0.5079 Acc: 0.7873
Epoch duration: 0 m 43s
Learning Rate = 0.001



Epoch 4/50
----------

train Loss: 0.5738 Acc: 0.7112
val Loss: 0.4834 Acc: 0.7892
Epoch duration: 0 m 43s
Learning Rate = 0.001



Epoch 5/50
----------

train Loss: 0.5513 Acc: 0.7357
val Loss: 0.4709 Acc: 0.8020
Epoch duration: 0 m 43s
Learning Rate = 0.0001



Epoch 6/50
----------

train Los

c:\Users\Adm\anaconda3\envs\computervision\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet169_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet169_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Only fully connected layer requires grad.
Saving model to folder notebooks/densenet\models\2024-11-18 15-27-40

Model DenseNet 169
Fully Trained = False
Dataset E:/Datasets/cor-splits\underperforming_cameras_60_v2
Learning Rate Epoch Schedule = 5
Learning Rate Gamma = 0.1
Optimizer = SGD
Batch Size = 16


Epoch 1/50
----------

train Loss: 0.7122 Acc: 0.4981
val Loss: 0.7255 Acc: 0.4735
Epoch duration: 0 m 45s
Learning Rate = 0.0001



Epoch 2/50
----------

train Loss: 0.6993 Acc: 0.5081
val Loss: 0.7029 Acc: 0.5333
Epoch duration: 0 m 44s
Learning Rate = 0.0001



Epoch 3/50
----------

train Loss: 0.6943 Acc: 0.5406
val Loss: 0.6801 Acc: 0.5765
Epoch duration: 0 m 44s
Learning Rate = 0.0001



Epoch 4/50
----------

train Loss: 0.6818 Acc: 0.5650
val Loss: 0.6669 Acc: 0.6069
Epoch duration: 0 m 44s
Learning Rate = 0.0001



Epoch 5/50
----------

train Loss: 0.6762 Acc: 0.5717
val Loss: 0.6516 Acc: 0.6314
Epoch duration: 0 m 48s
Learning Rate = 1e-05



Epoch 6/50
----------

train 

c:\Users\Adm\anaconda3\envs\computervision\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet169_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet169_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Only fully connected layer requires grad.
Saving model to folder notebooks/densenet\models\2024-11-18 15-49-09

Model DenseNet 169
Fully Trained = False
Dataset E:/Datasets/cor-splits\underperforming_cameras_60_v2
Learning Rate Epoch Schedule = 5
Learning Rate Gamma = 0.1
Optimizer = SGD
Batch Size = 16


Epoch 1/50
----------

train Loss: 0.7037 Acc: 0.5170
val Loss: 0.6890 Acc: 0.5529
Epoch duration: 0 m 45s
Learning Rate = 1e-05



Epoch 2/50
----------

train Loss: 0.6981 Acc: 0.5231
val Loss: 0.6822 Acc: 0.5686
Epoch duration: 0 m 44s
Learning Rate = 1e-05



Epoch 3/50
----------

train Loss: 0.6973 Acc: 0.5197
val Loss: 0.6822 Acc: 0.5598
Epoch duration: 0 m 44s
Learning Rate = 1e-05



Epoch 4/50
----------

train Loss: 0.6938 Acc: 0.5361
val Loss: 0.6744 Acc: 0.5725
Epoch duration: 0 m 44s
Learning Rate = 1e-05



Epoch 5/50
----------

train Loss: 0.6950 Acc: 0.5286
val Loss: 0.6737 Acc: 0.5824
Epoch duration: 0 m 44s
Learning Rate = 1.0000000000000002e-06



Epoch 6/50
-----